In [5]:
import branca
import csv
import folium
import sqlite3
from sqlalchemy import create_engine
import ipywidgets as widgets
import numpy as np
import pandas as pd
import draw_markers
import table_builder


In [6]:
with open('data/raw_data/Fake_Data_For_Draw_Markers.csv',encoding='ascii',errors='ignore') as csvfile:
    data = pd.read_csv(csvfile)
with open('data/collisions.csv',encoding='ascii',errors='ignore') as csvfile:
    collisions = pd.read_csv(csvfile)
with open('data/buildings.csv',encoding='ascii',errors='ignore') as csvfile:
    buildings = pd.read_csv(csvfile)
with open('data/radius.csv',encoding='ascii',errors='ignore') as csvfile:
    radius = pd.read_csv(csvfile)

In [8]:
before = np.where(radius['build_start_dt'] > radius['coll_dt'], 1, 0)
after = np.where(radius['build_end_dt'] < radius['coll_dt'], 1, 0)
during = np.where(((radius['build_end_dt'] >= radius['coll_dt']) & (radius['build_start_dt'] <= radius['coll_dt'])), 1, 0)
total = before + during + after
radius['coll_before'] = before
radius['coll_during'] = during
radius['coll_after'] = after
radius.to_csv('data/radius_with_bda.csv')

In [12]:


table_builder.create_table("my_db","data/")

OperationalError: unrecognized token: ":"

In [9]:
engine = create_engine('sqlite:///my_db')
connection = engine.connect()

In [4]:
connection = sqlite3.connect("my_db")
crsr = connection.cursor()


In [4]:
crsr.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = (crsr.fetchall())
# crsr.execute("SELECT * FROM buildings  where b_category == COMMERCIAL")
# buildings = crsr.fetchall()
# crsr.execute("SELECT bcount(*) FROM buildings WHERE b_category == 'COMMERCIAL'")
# crsr.fetchall()

crsr.execute("select * from buildings")
buildings_fields = list(map(lambda x: x[0], crsr.description))

crsr.execute("select * from collisions")
collisions_fields = list(map(lambda x: x[0], crsr.description))

crsr.execute("select * from radius_with_bda")
radius_fields = list(map(lambda x: x[0], crsr.description))

NameError: name 'crsr' is not defined

In [6]:
print("building fields:", buildings_fields)
print("collisions fields:", collisions_fields)
print("radius fields:", radius_fields)

building fields: ['text', 'b_id', 'b_category', 'b_value', 'b_issue_date', 'b_final_date', 'b_status', 'b_lat', 'b_long']
collisions fields: ['text', 'c_id', 'c_long', 'c_lat', 'c_datetime', 'c_ped', 'c_cyc', 'c_severity_code', 'c_severity_desc', 'c_veh']
radius fields: ['text', 'build_end_dt', 'build_id', 'build_lat', 'build_long', 'build_start_dt', 'coll_dt', 'coll_id', 'coll_lat', 'coll_long', 'radius']


In [20]:


crsr.execute("""
SELECT b.b_id as b_id, 
    b.b_lat as b_latitude,
    b.b_long as b_longitude, 
    count(*)/6 AS col_before, 
    count(*)/3 AS col_during, 
    count(*)/2 AS col_after 
FROM buildings b, radius r
WHERE b.b_id = r.build_id 
GROUP BY b.b_id""")
test = pd.DataFrame(crsr.fetchall())
test.columns = list(map(lambda x: x[0], crsr.description))

In [44]:

crsr.execute("""
SELECT b.b_id as b_id, 
    b.b_lat as b_latitude,
    b.b_long as b_longitude, 
    count(*)/6 AS col_before, 
    count(*)/3 AS col_during, 
    count(*)/2 AS col_after 
FROM buildings b, radius r
WHERE b.b_id = r.build_id AND b.b_category= 'COMMERCIAL'
GROUP BY b.b_id""")
test = pd.DataFrame(crsr.fetchall())

In [45]:
test

,0,1,2,3,4,5
0,6098854,47.64790943,-122.3381579,25,51,77
1,6120993,47.67747634,-122.3268962,30,61,92
2,6128401,47.6064785,-122.3313647,358,716,1074
3,6143649,47.61539704,-122.32709640000002,207,414,621
4,6148655,47.6064785,-122.3313647,358,716,1074
5,6155458,47.61712077,-122.331524,200,401,601
6,6162727,47.56122456,-122.381916,71,143,214
7,6164894,47.61070574,-122.3413759,264,528,792
8,6165135,47.72943418,-122.29156770000002,33,67,101
9,6170569,47.66838544,-122.37200690000002,64,128,193


In [23]:
test['b_latitude'] = test.b_latitude.astype(float)
test['b_longitude'] = test.b_longitude.astype(float)


In [22]:
draw_markers.place_maps(test)

In [6]:
def myInteraction(option, selection):
    querry = str("""SELECT b.b_id as b_id, b.b_lat as b_latitude, b.b_long as b_longitude, count(*)/6 AS col_before, count(*)/3 AS col_during, count(*)/2 AS col_after 
    FROM buildings b, radius r 
    WHERE b.b_id = r.build_id AND """ + my_option +  "=" + "'" + my_selection + "'"  + 
    " GROUP BY b.b_id")
    crsr.execute(querry)
    test = pd.DataFrame(crsr.fetchall())
    test.columns = list(map(lambda x: x[0], crsr.description))
 
    

In [50]:
my_option = "b.b_category"
my_selection = "COMMERCIAL"
querry = str("SELECT b.b_id as b_id, b.b_lat as b_latitude, b.b_long as b_longitude, count(*)/6 AS col_before, count(*)/3 AS col_during, count(*)/2 AS col_after FROM buildings b, radius r WHERE b.b_id = r.build_id AND " + my_option +  "=" + "'" + my_selection + "'"  + " GROUP BY b.b_id")
querry
crsr.execute(querry)


In [52]:
querry = str("""SELECT b.b_id as b_id, b.b_lat as b_latitude, b.b_long as b_longitude, count(*)/6 AS col_before, count(*)/3 AS col_during, count(*)/2 AS col_after 
    FROM buildings b, radius r 
    WHERE b.b_id = r.build_id AND """ + my_option +  "=" + "'" + my_selection + "'"  + 
    " GROUP BY b.b_id")
crsr.execute(querry)